In [3]:
import config
import pandas as pd
import psycopg2

Code acte consultation : « CNP »
Base tarif : 42,5 (Secteur 2 non OPTAM)
MPC : majoration du médecin spécialiste : 2,7 euros
Donc S1 et S2 OPTAM = 45,2
keep if strpos(v17,"CNP")
MCS: MCS : majoration de coordination pour les psychiatres : 5 euros

In [26]:
datesource_min = 2301
datesource_max = 2308
base_tarif = 42.5
mpc = 2.7
mcs = 5
tarif_seuil = base_tarif + mpc + mcs
tarif_seuil

50.2

In [10]:
sql = f"""
select * from ps p
join tarif t on t.ps_id =p.id
join tarif_date_source tds on tds.tarif_id =t.id 
join cabinet c on t.cabinet_id = c.id
join adresse_raw ar on c.adresse_raw_id = ar.id
join adresse_norm an on ar.adresse_norm_id = an.id
join ban b on an.ban_id  = b.id
where (t.profession_id =65 or t.profession_id =66)  # Faire mune jointure avec les diplomes vs profession j'ai l'info psy = 65 or 66 qqpart
and tds.date_source_id >= {datesource_min} and  tds.date_source_id <= {datesource_max}
"""

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# df = pd.read_sql(sql, config.connection_string)
df = pd.read_csv("C:/Users/conta/git-CVC/Skema/Part III/psychiatres.csv", low_memory=False)
df.describe()


,id,rule_nb,id.1,profession_id,mode_exercice_id,nature_id,convention_id,ps_id,cabinet_id,famille_acte_id,montant,borne_inf,borne_sup,montant_2,borne_inf_2,borne_sup_2,montant_imagerie,borne_inf_imagerie,borne_sup_imagerie,montant_anesthesie,borne_inf_anesthesie,borne_sup_anesthesie,montant_cec,borne_inf_cec,borne_sup_cec,tarif_id,date_source_id,id.2,telephone,adresse_raw_id,id.3,cp,adresse_norm_id,dept_id,id.4,numero,cp.1,osm_id,osm_score,ban_id,ban_score,source_id,lon,lat,score,dept_id.1,id.5,numero.1,code_postal,lon.1,lat.1,dept_id.2
count,62173.000000,60775.000000,6.217300e+04,62173.000000,0.0,62173.000000,62173.00000,62173.000000,62173.000000,12361.000000,62173.000000,62173.000000,62173.000000,0.0,0.0,0.0,12361.0,12361.0,12361.0,12361.000000,12361.000000,12361.000000,12361.0,12361.0,12361.0,6.217300e+04,62173.000000,62173.000000,4.659200e+04,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,58793.000000,62173.000000,4805.000000,4869.000000,6.217300e+04,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,62173.000000,6.217300e+04,61852.000000,62173.000000,62173.000000,62173.000000,62173.000000
mean,728301.521722,1.729165,2.002598e+07,65.013398,NaN,2.944108,1.89994,728301.521722,96303.205089,183.171426,35.301258,25.998795,34.364523,NaN,NaN,NaN,0.0,0.0,0.0,1.627053,0.570828,0.881887,0.0,0.0,0.0,2.002598e+07,2304.473823,96303.205089,4.343638e+08,77076.528573,77076.528573,52980.329484,77619.717273,53.948804,77619.717273,62.395846,52980.329484,14343.288866,0.812670,1.335062e+07,0.980543,2.092934,2.856080,46.675887,0.985698,53.948804,1.335062e+07,61.668402,52978.922812,2.856655,46.669992,53.948804
std,44495.514880,2.053991,2.606670e+06,0.114973,NaN,0.565625,0.99486,44495.514880,59623.065558,36.148945,44.621316,35.197314,46.984905,NaN,NaN,NaN,0.0,0.0,0.0,8.686618,5.203467,8.103884,0.0,0.0,0.0,2.606670e+06,2.294107,59623.065558,2.130620e+08,84888.944104,84888.944104,26293.534649,82095.580287,28.701967,82095.580287,164.191753,26293.534649,6978.001223,0.166871,7.277875e+06,0.045018,0.434307,2.671546,2.469264,0.033940,28.701967,7.277875e+06,179.944063,26292.988547,2.671045,2.525103,28.701967
min,648102.000000,1.000000,1.584330e+07,65.000000,NaN,1.000000,1.00000,648102.000000,63.000000,13.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.584330e+07,2301.000000,63.000000,1.300762e+08,60.000000,60.000000,1000.000000,60.000000,1.000000,60.000000,1.000000,1000.000000,6348.000000,0.000000,2.983000e+03,0.671429,1.000000,-4.530457,0.000000,0.571429,1.000000,2.983000e+03,1.000000,1000.000000,-4.530457,0.000000,1.000000
25%,693181.000000,1.000000,1.751675e+07,65.000000,NaN,3.000000,1.00000,693181.000000,50489.000000,201.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.751675e+07,2302.000000,50489.000000,2.412315e+08,25753.000000,25753.000000,33000.000000,26729.000000,33.000000,26729.000000,7.000000,33000.000000,9017.000000,0.700000,7.292780e+06,0.987500,2.000000,1.445443,44.015019,0.987500,33.000000,7.292780e+06,6.000000,33000.000000,1.445443,44.013227,33.000000
50%,730155.000000,1.000000,1.996902e+07,65.000000,NaN,3.000000,1.00000,730155.000000,91943.000000,201.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.996902e+07,2304.000000,91943.000000,4.685052e+08,48614.000000,48614.000000,59260.000000,50162.000000,59.000000,50162.000000,20.000000,59260.000000,12648.000000,0.850000,1.474418e+07,1.000000,2.000000,2.381545,47.249092,1.000000,59.000000,1.474418e+07,18.000000,59260.000000,2.381523,47.245400,59.000000
75%,768750.000000,1.000000,2.264998e+07,65.000000,NaN,3.000000,3.00000,768750.000000,135649.000000,201.000000,70.000000,50.200000,75.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,2.264998e+07,2306.000000,135649.000000,5.611559e+08,80132.000000,80132.000000,75012.000000,85014.000000,75.000000,85014.000000,55.000000,75012.000000,1522

In [11]:
# Renommer les colonnes
df.rename(columns={
    'genre': 'gender',
    'prenom': 'prénom',
    'code': 'codeccamdelacte',
    'nature_id': 'naturedexercice',
    'convention_id': 'convention',
    'code_insee': 'codeinsee',
    'code_postal': 'matchcp',
    'montant': 'montantgénéralementconstaté',
    'borne_inf': 'borneinférieuredumontant',
    'borne_sup': 'bornesupérieuredumontant',
    'option_contrat': 'optioncontratdaccèsauxsoins',
    'dept_id': 'dep'
}, inplace=True)

# Colonnes à conserver
df = df[['ps_id', 'gender', 'nom', 'prénom', 'codeccamdelacte', 'naturedexercice', 'convention',
         'codeinsee', 'montantgénéralementconstaté', 'borneinférieuredumontant', 'bornesupérieuredumontant',
         'adresse_id', 'date_source_id', 'optioncontratdaccèsauxsoins', 'matchcp', 'dep']]
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep
0,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2301,False,6000,6
1,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2302,False,6000,6
2,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2303,False,6000,6
3,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2304,False,6000,6
4,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2305,False,6000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62168,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2304,False,45500,45
62169,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2305,False,45500,45
62170,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2306,False,45500,45
62171,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2307,False,45500,45


In [12]:
# Clé combinée
df['b'] = df['ps_id'].astype(str) + df['adresse_id'].astype(str)
df

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b
0,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2301,False,6000,6,64990806088_4790_00012
1,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2302,False,6000,6,64990806088_4790_00012
2,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2303,False,6000,6,64990806088_4790_00012
3,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2304,False,6000,6,64990806088_4790_00012
4,649908,F,BAUDOIN,HELENE,CNP+MPC+MCS,3,1,6088,0.0,0.0,0.0,06088_4790_00012,2305,False,6000,6,64990806088_4790_00012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62168,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2304,False,45500,45,72957945155_0412_00001
62169,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2305,False,45500,45,72957945155_0412_00001
62170,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2306,False,45500,45,72957945155_0412_00001
62171,729579,H,AKHRIF,MEHDI,CNP+MPC+MCS,2,1,45155,0.0,0.0,0.0,45155_0412_00001,2307,False,45500,45,72957945155_0412_00001


In [16]:
nb_ps_id = df["ps_id"].nunique()
nb_ps_id  # 5439


5439

In [17]:
nb_b = df["b"].nunique()
nb_b  # 5674

5674

In [18]:

df = df.sort_values(by='ps_id')
df


,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,64810201091_0331_00539
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,64810201091_0331_00539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61230,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2303,False,13009,13,79963413209_4629_00002
61235,799634,F,VINCENTI,ODILE,CNP+MPC+MCS,3,1,13209,0.0,0.0,0.0,13209_4629_00002,2308,False,13009,13,79963413209_4629_00002
15325,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2308,False,31470,31,79973131188_2noilo_00003
15324,799731,F,GAUDIN,MAUD,CNP+MPC+MCS,3,1,31188,0.0,0.0,0.0,31188_2noilo_00003,2307,False,31470,31,79973131188_2noilo_00003


In [24]:
df['unique'] = df.groupby('ps_id')['adresse_id'].transform('nunique')
df['un'] = df['unique']
pd.set_option('display.max_rows', 100)
df.head(100)

,ps_id,gender,nom,prénom,codeccamdelacte,naturedexercice,convention,codeinsee,montantgénéralementconstaté,borneinférieuredumontant,bornesupérieuredumontant,adresse_id,date_source_id,optioncontratdaccèsauxsoins,matchcp,dep,b,unique,un
31684,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2302,False,1200,1,64810201091_0331_00539,1,1
31686,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2304,False,1200,1,64810201091_0331_00539,1,1
31687,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2305,False,1200,1,64810201091_0331_00539,1,1
31688,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2306,False,1200,1,64810201091_0331_00539,1,1
31689,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2307,False,1200,1,64810201091_0331_00539,1,1
31690,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2308,False,1200,1,64810201091_0331_00539,1,1
31683,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2301,False,1200,1,64810201091_0331_00539,1,1
31685,648102,H,BOULAND,PHILIPPE,CNP+MPC+MCS,3,1,1033,83.5,50.2,83.5,01091_0331_00539,2303,False,1200,1,64810201091_0331_00539,1,1
31715,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2301,False,1200,1,64810701091_0331_00539,1,1
31716,648107,F,SPIRIDONOVA,LUBA,ALQP0030,3,1,1033,0.0,0.0,0.0,01091_0331_00539,2302,False,1200,1,64810701091_0331_00539,1,1


In [34]:
# Comptages globaux
df['NB_total'] = df.groupby('dep')['ps_id'].transform('nunique')
df = df.sort_values(by='dep')
df["NB_total"].unique()

array([  14,   24,   10,   11,    6,  188,    8,    7,    5,   15,  346,
         40,    3,   34,   45,   28,   17,   38,   20,   26,    9,   56,
         53,  204,  289,  164,   86,   66,  118,   22,   49,  113,    2,
         58,   35,    4,   64,   41,   44,  122,   19,   32,   81,   72,
         12,   33,  145,  272,   57, 1016,   76,  102,   13,   89,   39,
         18,   29,  132,   52], dtype=int64)